# BayerNorm

<img src="./batchnorm_1d.png" width="300" height="400">  

<img src="./batchnorm_2d.png" width="300" height="400">

<img src="./batchnorm_eq.png" width="200" height="100">

思路：MLP中在特征维度归一化，CNN中在channel维度归一化

In [ ]:
class BatchNorm(nn.Module):
    # num_features：完全连接层的输出数量或卷积层的输出通道数。
    def __init__(self, num_features, num_dims):
        super().__init__()
        if num_dims == 2: # 全连接层
            shape = (1, num_features)
        else:             # 卷积层
            shape = (1, num_features, 1, 1)
        
        # 参与求梯度和迭代的拉伸和偏移参数，分别初始化成1和0
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        
        # 非模型参数的变量初始化为0和1
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.ones(shape)

    def batch_norm(self, X, gamma, beta, moving_mean, moving_var, eps, momentum):
        if not torch.is_grad_enabled():
            # 如果是在预测模式下，直接使用传入的移动平均所得的均值和方差
            X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
        else:
            assert len(X.shape) in (2, 4)
            if len(X.shape) == 2:
                # 使用全连接层的情况，计算特征维上的均值和方差
                mean = X.mean(dim=0)                                       # (num_features,)
                var = ((X - mean) ** 2).mean(dim=0)                        # (num_features,)
            else:
                # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。
                mean = X.mean(dim=(0, 2, 3), keepdim=True)                # (1,num_features,1,1) 保持X的形状，以便后面可以做广播运算
                var = ((X - mean) ** 2).mean(dim=(0, 2, 3), keepdim=True) # (1,num_features,1,1)
                
            # 训练模式下，用当前的均值和方差做标准化
            X_hat = (X - mean) / torch.sqrt(var + eps)
            
            # 更新移动平均的均值和方差
            moving_mean = momentum * moving_mean + (1.0 - momentum) * mean
            moving_var = momentum * moving_var + (1.0 - momentum) * var
            
        Y = gamma * X_hat + beta  # 缩放和移位
        return Y, moving_mean.data, moving_var.data

    def forward(self, X):
        # 如果X不在内存上，将moving_mean和moving_var，复制到X所在显存上
        if self.moving_mean.device != X.device:
            self.moving_mean = self.moving_mean.to(X.device)
            self.moving_var = self.moving_var.to(X.device)
       
        # 保存更新过的moving_mean和moving_var
        Y, self.moving_mean, self.moving_var = self.batch_norm(
            X, self.gamma, self.beta, self.moving_mean,
            self.moving_var, eps=1e-5, momentum=0.9
        )
        return Y


# LatchNorm

<img src="./layernorm_1d.png" width="300" height="400">  

<img src="./layernorm_2d.png" width="300" height="400">  

<img src="./batchnorm_eq.png" width="200" height="100">

思路：对一个batch中的输入归一化，对每个token的feature进行归一化，缩放到了统一的尺度，随后对每个特征维度施加统一的拉伸和偏移

LayerNorm 层有两种放置方式：Pre Norm和Post Norm  
<img src="./two_layernorm.png" width="300" height="400">  

同一设置之下，Pre Norm结构往往更容易训练，但最终效果通常不如Post Norm
1. Pre Norm 更容易训练好理解，因为它的恒等路径更突出
2. Pre Norm 中多层叠加的结果更多是增加宽度而不是深度，层数越多，这个层就越“虚”，这是因为 Pre Norm 结构无形地增加了模型的宽度而降低了模型的深度，而我们知道深度通常比宽度更重要，所以是无形之中的降低深度导致最终效果变差了。而 Post Norm 刚刚相反，它每Norm一次就削弱一次恒等分支的权重，所以 Post Norm 反而是更突出残差分支的，因此Post Norm中的层数更加有分量，起到了作用，一旦训练好之后效果更优。

过去 BERT 主流的时代往往使用 Post Norm，现在 GPT 时代模型规模都很大，因此大多用 Pre Norm 来稳定训练

# RMSNorm

<img src="./RMSNorm.png" width="500" height="100">  

 思路：layernorm的变体，简单看作均值为0，减少了计算均值的步骤，减少了运算，也没有拉升操作，减少了参数

In [ ]:
class LlamaRMSNorm(nn.Module):
    def __init__(self, hidden_size, eps=1e-6):
        """
        LlamaRMSNorm is equivalent to T5LayerNorm
        """
        super().__init__()
        self.weight = nn.Parameter(torch.ones(hidden_size))
        self.variance_epsilon = eps

    def forward(self, hidden_states):
        input_dtype = hidden_states.dtype
        hidden_states = hidden_states.to(torch.float32)
        variance = hidden_states.pow(2).mean(-1, keepdim=True)
        hidden_states = hidden_states * torch.rsqrt(variance + self.variance_epsilon)
        return self.weight * hidden_states.to(input_dtype)